In [2]:
from zse.development import *
from zse.utilities import *
from zse.collections import *
from zse.ring_utilities import *

from ase.visualize import view

import numpy as np

# First, Get Rings
Get all the valid and invalid rings based on the d2 validation method. \
Visualize both sets of rings, and ensure that there are no valid rings in the invalid list.

In [18]:
code = 'RWY'
z = framework(code)
ts,tm,tinds = get_tsites(code)
vp,ip,vr,ir = get_trings(z,tinds[0],code,'d2')
print([int(len(p)/2) for p in vp if len(p)==16])
view(vr)
view(ir)

[8]


# Second, Make a Feature List
Once we are comfortable with which rings were returned by the d2 validation method, we can make a feature list. \
The y values will be 1 for valid ring and 2 for invalid ring.

In [19]:
def data_update(code,paths,size,y):
    z = framework(code)
    ring_sizes = get_ring_sizes(code)*2
    max_ring = max(ring_sizes)
    features = []
    yvals = []
    for i,p in enumerate(paths):
        if len(p) == size*2:
            repeat = atoms_to_graph(z,p[0],max_ring)[2]
            atoms = z.repeat(repeat)
            atoms,trans = center(atoms,p[0])
            n = int(len(p)/2)
            distances = []
            for x in range(1,n,2):
                dist = atoms.get_distance(p[x],p[x+n])
                distances.append(dist)
                distances.sort(reverse=True)
            features.append(distances)
            yvals.append([y])
    return features,yvals

In [20]:
# first do the valid paths
size = 8
features,yvals = data_update(code,vp,size,1)

f = open('8mrx.csv','ab')
np.savetxt(f,features,delimiter=',')
f.close()

f = open('8mry.csv','ab')
np.savetxt(f,yvals,delimiter=',')
f.close()

# next the invalid paths
features,yvals = data_update(code,ip,size,0)

f = open('8mrx.csv','ab')
np.savetxt(f,features,delimiter=',')
f.close()

f = open('8mry.csv','ab')
np.savetxt(f,yvals,delimiter=',')
f.close()

# Test the Data in our Neural Net

In [7]:
import numpy as np
import torch
from torch.autograd import Variable
import torch.nn.functional as F
import torch.utils.data as Data

from sklearn.preprocessing import StandardScaler    
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report

In [21]:
x_data = np.loadtxt('8mrx.csv',delimiter=',',dtype=float)
y_data = np.loadtxt('8mry.csv',delimiter=',',dtype=float,ndmin=2)
scaler = StandardScaler()
x_data = scaler.fit_transform(x_data)


x_data = torch.tensor(x_data)
x_data = Variable(x_data)

y_data = torch.tensor(y_data)
y_data = Variable(y_data)

In [22]:
torch.manual_seed(1);

# Define the network class

class Net(torch.nn.Module):
    def __init__(self, n_feature, n_hidden, n_output):
        super(Net, self).__init__()
        self.hidden = torch.nn.Linear(n_feature, n_hidden)
        self.predict = torch.nn.Linear(n_hidden, n_output)
        
    def forward(self,x):
        x = torch.sigmoid(self.hidden(x))
        x = self.predict(x)
        return x
    
# Define our network

net = Net(n_feature = len(x_data[0]), n_hidden = 12, n_output=1)

# Define our optimizer

optimizer = torch.optim.Rprop(net.parameters(), lr = .0002)

# Define our loss function (mean squared error)
loss_func = torch.nn.MSELoss()

In [23]:
for t in range(1000):
    prediction = net(x_data.float())

    loss = loss_func(prediction,y_data.float())
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
print(loss.item())

8.29960572445998e-06


In [24]:
print('actual_y \t y_prediction')
for realy,y in zip(y_data,prediction):
    realy = realy.item()
    y = y.item()
    print('{0} \t\t {1}'.format(abs(round(realy)),abs(round(y,2))))

actual_y 	 y_prediction
1 		 1.0
1 		 1.0
0 		 0.0
0 		 0.0
0 		 0.0
0 		 0.0
0 		 0.0
0 		 0.0
0 		 0.0
0 		 0.0
0 		 0.0
0 		 0.0
0 		 0.0
0 		 0.0
0 		 0.0
0 		 0.0
0 		 0.0
0 		 0.0
0 		 0.0
0 		 0.0
0 		 0.0
0 		 0.0
0 		 0.0
0 		 0.0
0 		 0.0
0 		 0.0
0 		 0.0
0 		 0.0
0 		 0.0
0 		 0.0
0 		 0.0
0 		 0.0
0 		 0.0
0 		 0.0
0 		 0.0
0 		 0.0
0 		 0.0
0 		 0.0
0 		 0.0
0 		 0.0
1 		 1.0
1 		 1.0
1 		 1.0
1 		 1.0
0 		 0.0
0 		 0.0
0 		 0.0
0 		 0.0
0 		 0.0
0 		 0.0
0 		 0.0
0 		 0.0
0 		 0.0
0 		 0.0
0 		 0.0
0 		 0.0
0 		 0.0
0 		 0.0
0 		 0.0
0 		 0.0
0 		 0.0
0 		 0.0
0 		 0.0
0 		 0.0
0 		 0.0
0 		 0.0
0 		 0.0
0 		 0.0
0 		 0.0
0 		 0.0
0 		 0.0
0 		 0.0
1 		 1.0
1 		 1.0
1 		 1.0
1 		 1.0
1 		 1.0
1 		 1.0
0 		 0.0
0 		 0.0
0 		 0.0
0 		 0.0
0 		 0.0
0 		 0.0
1 		 1.0
1 		 1.0
1 		 1.0
1 		 1.0
1 		 1.0
1 		 1.0
1 		 1.0
1 		 1.0
1 		 1.0
1 		 1.0
1 		 1.0
1 		 1.0
1 		 1.0
1 		 1.0
1 		 1.0
1 		 1.0
0 		 0.0
0 		 0.0
0 		 0.0
0 		 0.0
0 		 0.0
0 		 0.0
0 		 0.0
0 		 0.0
0 		

# Let's Test a Framework We Didn't Train With

In [28]:
code = 'RWR'
z = framework(code)
ts,tm,tinds = get_tsites(code)
vp,ip,vr,ir = get_trings(z,tinds[0],code,'d2')
view(vr)

In [29]:
# Check the valid rings, these should all be 1
size = 8
features,yvals = data_update(code,vp,size,1)

features = scaler.transform(features)
x_pred = torch.tensor(features)
x_pred = Variable(x_pred)

n_pred = net(x_pred.float())

i=1
for y in n_pred:
    y = y.item()
    print('{0} \t {1} \t {2}'.format(i,abs(round(y,2)),abs(round(y,0))))
    i+=1

1 	 1.0 	 1.0
2 	 1.0 	 1.0


In [30]:
# Check the invalid rings, these should all be 0
size = 8
features,yvals = data_update(code,ip,size,1)

features = scaler.transform(features)
x_pred = torch.tensor(features)
x_pred = Variable(x_pred)

n_pred = net(x_pred.float())

i=1
for y in n_pred:
    y = y.item()
    print('{0} \t {1} \t {2}'.format(i,abs(round(y,2)),abs(round(y,0))))
    i+=1

1 	 0.07 	 0.0
2 	 0.32 	 0.0
3 	 0.07 	 0.0
4 	 0.07 	 0.0
5 	 0.03 	 0.0
6 	 0.07 	 0.0
7 	 0.32 	 0.0
8 	 0.03 	 0.0
9 	 0.03 	 0.0


All the invalid rings returned a value less than 0.5. This is decent, but not terrific. 